# Generate Weekday/Hour Offset Mapping

This code ensures that a "trading day" starts at 17:00 on one day and runs through 16:59 on the following day.

Oanda trading hours:  Sunday 17:05 through Friday 16:59 (New York time)

## Load useful libraries

In [1]:
import pandas as pd
import datetime
import pytz
import os

In [2]:
import boilerplate
from timeseries.models import Interval, Instrument, PriceType, Candlestick
from utilities.make_dataframe_from_db import make_candlestick_dataframe
from utilities.date_and_time_related_calculations import compute_datetime_information

In [3]:
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

## User settings

In [4]:
#
# the following criteria rely on the database having candlestick values for them:
#
price_type_name = 'mid'
instrument_name = 'EUR/USD'
interval_name = 'Minute'

In [5]:
tz = pytz.timezone('US/Eastern')
output_directory = os.environ['APP_HOME'] + '/badassdatascience/forecasting/deep_learning/output'

In [6]:
print(output_directory)

/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning/output


## Generate the mapping from the data

In [ ]:
df_from_data = make_candlestick_dataframe(price_type_name, instrument_name, interval_name)
df_from_data = compute_datetime_information(df_from_data, tz)

df_shifted_weekday_from_data = df_from_data[['weekday_tz', 'hour_tz']].drop_duplicates().sort_values(by = ['weekday_tz', 'hour_tz']).copy()

df_shifted_weekday_from_data['weekday_shifted'] = (
    df_shifted_weekday_from_data['weekday_tz'].shift(-7)
    .fillna(0.)
    .astype('int32')
)

df_shifted_weekday_from_data['weekday_shifted'] = [4 if x == 6 else x for x in df_shifted_weekday_from_data['weekday_shifted']]

print(df_shifted_weekday_from_data.to_string())

## Generate the mapping manually

By creating the mapping twice we help QA along:

In [ ]:
hour_list = []
weekday_list = []
shifted_list = []

shifted_list.extend([0] * 17)

for i in range(0, 4):
    weekday_list.extend([i] * 24)
    hour_list.extend(sorted(list(range(0, 24))))

    if i >= 1:
        shifted_list.extend([i] * 24)

shifted_list.extend([4] * 24)

weekday_list.extend([4] * 17)
hour_list.extend(sorted(list(range(0, 17))))

weekday_list.extend([6] * 7)
hour_list.extend(sorted(list(range(17, 24))))
shifted_list.extend([0] * 7)

df_shifted_weekday_manually_constructed = pd.DataFrame({'weekday_tz' : weekday_list, 'hour_tz' : hour_list, 'weekday_shifted' : shifted_list})

## QA

#### Test for dataframe equality

In [ ]:
for column_name in ['weekday_tz', 'hour_tz', 'weekday_shifted']:
    print(list(df_shifted_weekday_from_data[column_name]) == list(df_shifted_weekday_manually_constructed[column_name]))
    print(min([int(i) for i in [x == y for x, y in zip(df_shifted_weekday_from_data[column_name], df_shifted_weekday_manually_constructed[column_name])]]))

#### Aggregations

In [ ]:
df_test = df_shifted_weekday_manually_constructed.copy()
df_test['dummy_variable'] = 1

(
    df_test
    .groupby(['weekday_tz'])
    ['dummy_variable']
    .agg('count')
    .reset_index()
)

In [ ]:
df_agg_hour_tz = (
    df_test
    .groupby(['hour_tz'])
    ['dummy_variable']
    .agg('count')
    .reset_index()
)

df_agg_hour_tz

In [ ]:
print(min(df_agg_hour_tz['dummy_variable']), max(df_agg_hour_tz['dummy_variable']))

In [ ]:
(
    df_test
    .groupby(['weekday_shifted'])
    ['dummy_variable']
    .agg('count')
    .reset_index()
)

## Save mapping

In [ ]:
df_shifted_weekday_manually_constructed.to_csv(output_directory + '/df_weekday_shift_lookup_table.csv', index=False)